In [1]:
import torch
from PIL import Image, ImageOps
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os

from torchvision import transforms

In [2]:
model_path = "/workspace/nabang1010/STEAM/LeGiaBach_STEAM/weight/checkpoints_efficientNetB4/efficientNet_b4_jtit_f1_0.927.pt"
model = torch.jit.load(model_path)
model.eval() 
model.to("cuda")

RecursiveScriptModule(
  original_name=Model_classify
  (model_backbone): RecursiveScriptModule(
    original_name=EfficientNet
    (features): RecursiveScriptModule(
      original_name=Sequential
      (0): RecursiveScriptModule(
        original_name=Conv2dNormActivation
        (0): RecursiveScriptModule(original_name=Conv2d)
        (1): RecursiveScriptModule(original_name=BatchNorm2d)
        (2): RecursiveScriptModule(original_name=SiLU)
      )
      (1): RecursiveScriptModule(
        original_name=Sequential
        (0): RecursiveScriptModule(
          original_name=MBConv
          (block): RecursiveScriptModule(
            original_name=Sequential
            (0): RecursiveScriptModule(
              original_name=Conv2dNormActivation
              (0): RecursiveScriptModule(original_name=Conv2d)
              (1): RecursiveScriptModule(original_name=BatchNorm2d)
              (2): RecursiveScriptModule(original_name=SiLU)
            )
            (1): RecursiveScriptMod

In [3]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
transform_test = transforms.Compose([ transforms.Resize((224,224)),transforms.ToTensor(),normalize])

In [4]:
TEST_RAW = "/workspace/nabang1010/STEAM/LeGiaBach_STEAM/DATA/dataset/test_png"
list_test_images = os.listdir(TEST_RAW)
submission = pd.DataFrame(columns=["image_id", "class_id"])
for image_file in list_test_images:
    img = Image.open(os.path.join(TEST_RAW, image_file))
    img = ImageOps.exif_transpose(img)
    img = img.convert('RGB')

    img = transform_test(img)
    img = img.unsqueeze(0)
    img = img.to("cuda")
    output = model(img)
    _, pred = torch.max(output, 1)  
    submission = submission.append({"image_id": image_file[:-4], "class_id": pred.item()}, ignore_index=True)
    # clear cuda cache
    # torch.cuda.empty_cache()
    

    

In [5]:
submission.to_csv("/workspace/nabang1010/STEAM/LeGiaBach_STEAM/src/EfficientNetB4/submission.csv", index=False)